In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import bitsandbytes
from os import environ

/opt/conda/envs/py_3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
environ["HIP_VISIBLE_DEVICES"]="0"

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    count = torch.cuda.device_count()

__CUDNN VERSION: 3001000
__Number CUDA Devices: 2


In [3]:
model_name = "microsoft/Phi-3-mini-128k-instruct"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    device_map="cuda",
    load_in_8bit=True,
    trust_remote_code=True
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.57s/it]


In [6]:
system_prompt = "You are a programmer that takes in sparse descriptions of a psychology experiment from a scientific paper and generates a PsychoPy Psyexp experiment file that can execute the described experiment."
query = "In this task participants are required to memorise and recall number series in order. Participants start out with three 3-digit sequences. If participants correctly recall 2 out of 3 three sequences, they progress to 4-digit sequence trials and so on. If participants respond incorrectly on 2/3 trials the experiment terminantes. This experiment is based on the original digit span experiment by Jacobs (1887)."

In [7]:
system_prompt = "Given a PsychoPy Psyexp file, explain in english what the experiment is doing."
query = '''<?xml version="1.0" ?>
<PsychoPy2experiment encoding="utf-8" version="2023.2.2">
  <Settings>
    <Param val="3" valType="str" updates="None" name="Audio latency priority"/>
    <Param val="ptb" valType="str" updates="None" name="Audio lib"/>
    <Param val="" valType="str" updates="None" name="Completed URL"/>
    <Param val="auto" valType="str" updates="None" name="Data file delimiter"/>
    <Param val="u'data/%s_%s_%s' % (expInfo['participant'], expName, expInfo['date'])" valType="code" updates="None" name="Data filename"/>
    <Param val="True" valType="bool" updates="None" name="Enable Escape"/>
    <Param val="" valType="str" updates="None" name="End Message"/>
    <Param val="{'participant': 'f&quot;{randint(0, 999999):06.0f}&quot;', 'group': '[&quot;A&quot;, &quot;B&quot;]'}" valType="code" updates="None" name="Experiment info"/>
    <Param val="True" valType="bool" updates="None" name="Force stereo"/>
    <Param val="True" valType="bool" updates="None" name="Full-screen window"/>
    <Param val="" valType="str" updates="None" name="HTML path"/>
    <Param val="" valType="str" updates="None" name="Incomplete URL"/>
    <Param val="testMonitor" valType="str" updates="None" name="Monitor"/>
    <Param val="[]" valType="list" updates="None" name="Resources"/>
    <Param val="False" valType="bool" updates="None" name="Save csv file"/>
    <Param val="False" valType="bool" updates="None" name="Save excel file"/>
    <Param val="False" valType="bool" updates="None" name="Save hdf5 file"/>
    <Param val="True" valType="bool" updates="None" name="Save log file"/>
    <Param val="True" valType="bool" updates="None" name="Save psydat file"/>
    <Param val="True" valType="bool" updates="None" name="Save wide csv file"/>
    <Param val="1" valType="num" updates="None" name="Screen"/>
    <Param val="True" valType="bool" updates="None" name="Show info dlg"/>
    <Param val="False" valType="bool" updates="None" name="Show mouse"/>
    <Param val="height" valType="str" updates="None" name="Units"/>
    <Param val="" valType="str" updates="None" name="Use version"/>
    <Param val="[1440, 900]" valType="list" updates="None" name="Window size (pixels)"/>
    <Param val="none" valType="str" updates="None" name="backgroundFit"/>
    <Param val="" valType="str" updates="None" name="backgroundImg"/>
    <Param val="avg" valType="str" updates="None" name="blendMode"/>
    <Param val="{'thisRow.t': 'priority.CRITICAL', 'expName': 'priority.LOW'}" valType="dict" updates="None" name="colPriority"/>
    <Param val="$[0,0,0]" valType="color" updates="None" name="color"/>
    <Param val="rgb" valType="str" updates="None" name="colorSpace"/>
    <Param val="100.1.1.1" valType="str" updates="None" name="elAddress"/>
    <Param val="FILTER_LEVEL_2" valType="str" updates="None" name="elDataFiltering"/>
    <Param val="FILTER_LEVEL_OFF" valType="str" updates="None" name="elLiveFiltering"/>
    <Param val="EYELINK 1000 DESKTOP" valType="str" updates="None" name="elModel"/>
    <Param val="ELLIPSE_FIT" valType="str" updates="None" name="elPupilAlgorithm"/>
    <Param val="PUPIL_AREA" valType="str" updates="None" name="elPupilMeasure"/>
    <Param val="1000" valType="num" updates="None" name="elSampleRate"/>
    <Param val="False" valType="bool" updates="None" name="elSimMode"/>
    <Param val="RIGHT_EYE" valType="str" updates="None" name="elTrackEyes"/>
    <Param val="PUPIL_CR_TRACKING" valType="str" updates="None" name="elTrackingMode"/>
    <Param val="bilingual_stroop" valType="str" updates="None" name="expName"/>
    <Param val="on Sync" valType="str" updates="None" name="exportHTML"/>
    <Param val="None" valType="str" updates="None" name="eyetracker"/>
    <Param val="127.0.0.1" valType="str" updates="None" name="gpAddress"/>
    <Param val="4242" valType="num" updates="None" name="gpPort"/>
    <Param val="ioHub" valType="str" updates="None" name="keyboardBackend"/>
    <Param val="exp" valType="code" updates="None" name="logging level"/>
    <Param val="('MIDDLE_BUTTON',)" valType="list" updates="None" name="mgBlink"/>
    <Param val="CONTINUOUS" valType="str" updates="None" name="mgMove"/>
    <Param val="0.5" valType="num" updates="None" name="mgSaccade"/>
    <Param val="0.6" valType="num" updates="None" name="plConfidenceThreshold"/>
    <Param val="True" valType="bool" updates="None" name="plPupilCaptureRecordingEnabled"/>
    <Param val="" valType="str" updates="None" name="plPupilCaptureRecordingLocation"/>
    <Param val="127.0.0.1" valType="str" updates="None" name="plPupilRemoteAddress"/>
    <Param val="50020" valType="num" updates="None" name="plPupilRemotePort"/>
    <Param val="1000" valType="num" updates="None" name="plPupilRemoteTimeoutMs"/>
    <Param val="False" valType="bool" updates="None" name="plPupillometryOnly"/>
    <Param val="psychopy_iohub_surface" valType="str" updates="None" name="plSurfaceName"/>
    <Param val="time" valType="str" updates="None" name="sortColumns"/>
    <Param val="" valType="str" updates="None" name="tbLicenseFile"/>
    <Param val="" valType="str" updates="None" name="tbModel"/>
    <Param val="60" valType="num" updates="None" name="tbSampleRate"/>
    <Param val="" valType="str" updates="None" name="tbSerialNo"/>
    <Param val="pyglet" valType="str" updates="None" name="winBackend"/>
  </Settings>
  <Routines>
    <Routine name="intro">
      <RoutineSettingsComponent name="intro" plugin="None">
        <Param val="none" valType="str" updates="None" name="backgroundFit"/>
        <Param val="" valType="str" updates="None" name="backgroundImg"/>
        <Param val="$[0,0,0]" valType="color" updates="None" name="color"/>
        <Param val="rgb" valType="str" updates="None" name="colorSpace"/>
        <Param val="" valType="str" updates="constant" name="desc"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="intro" valType="code" updates="None" name="name"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="constant" name="skipIf"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="False" valType="bool" updates="None" name="useWindowParams"/>
      </RoutineSettingsComponent>
      <ImageComponent name="background" plugin="None">
        <Param val="center" valType="str" updates="constant" name="anchor"/>
        <Param val="$[1,1,1]" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="False" valType="bool" updates="constant" name="flipHoriz"/>
        <Param val="False" valType="bool" updates="constant" name="flipVert"/>
        <Param val="background.png" valType="file" updates="constant" name="image"/>
        <Param val="linear" valType="str" updates="constant" name="interpolate"/>
        <Param val="" valType="str" updates="constant" name="mask"/>
        <Param val="background" valType="code" updates="None" name="name"/>
        <Param val="" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="(2, 1.5)" valType="list" updates="constant" name="size"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="128" valType="num" updates="constant" name="texture resolution"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
      </ImageComponent>
      <TextComponent name="text" plugin="None">
        <Param val="white" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="None" valType="str" updates="constant" name="flip"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.03" valType="num" updates="constant" name="letterHeight"/>
        <Param val="text" valType="code" updates="None" name="name"/>
        <Param val="" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="This is a demo of a bilingual Stroop task. &amp;#10;&amp;#10;This example also shows how to achieve a counterbalanced design. The demo presents two blocks, an English block and a Maori block, group A will see English words first and group B will see Maori words first. &amp;#10;&amp;#10;Press space to start." valType="str" updates="constant" name="text"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
        <Param val="" valType="num" updates="constant" name="wrapWidth"/>
      </TextComponent>
      <KeyboardComponent name="start_resp" plugin="None">
        <Param val="'space'" valType="list" updates="constant" name="allowedKeys"/>
        <Param val="" valType="str" updates="constant" name="correctAns"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="True" valType="bool" updates="constant" name="discard previous"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="True" valType="bool" updates="constant" name="forceEndRoutine"/>
        <Param val="start_resp" valType="code" updates="None" name="name"/>
        <Param val="press" valType="str" updates="constant" name="registerOn"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="last key" valType="str" updates="constant" name="store"/>
        <Param val="False" valType="bool" updates="constant" name="storeCorrect"/>
        <Param val="True" valType="bool" updates="constant" name="syncScreenRefresh"/>
      </KeyboardComponent>
    </Routine>
    <Routine name="instructions">
      <RoutineSettingsComponent name="instructions" plugin="None">
        <Param val="none" valType="str" updates="None" name="backgroundFit"/>
        <Param val="" valType="str" updates="None" name="backgroundImg"/>
        <Param val="$[0,0,0]" valType="color" updates="None" name="color"/>
        <Param val="rgb" valType="str" updates="None" name="colorSpace"/>
        <Param val="" valType="str" updates="constant" name="desc"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="instructions" valType="code" updates="None" name="name"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="constant" name="skipIf"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="False" valType="bool" updates="None" name="useWindowParams"/>
      </RoutineSettingsComponent>
      <ImageComponent name="background_2" plugin="None">
        <Param val="center" valType="str" updates="constant" name="anchor"/>
        <Param val="$[1,1,1]" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="False" valType="bool" updates="constant" name="flipHoriz"/>
        <Param val="False" valType="bool" updates="constant" name="flipVert"/>
        <Param val="background.png" valType="file" updates="constant" name="image"/>
        <Param val="linear" valType="str" updates="constant" name="interpolate"/>
        <Param val="" valType="str" updates="constant" name="mask"/>
        <Param val="background_2" valType="code" updates="None" name="name"/>
        <Param val="" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="(2, 1.5)" valType="list" updates="constant" name="size"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="128" valType="num" updates="constant" name="texture resolution"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
      </ImageComponent>
      <TextComponent name="Q" plugin="None">
        <Param val="white" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="None" valType="str" updates="constant" name="flip"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.05" valType="num" updates="constant" name="letterHeight"/>
        <Param val="Q" valType="code" updates="None" name="name"/>
        <Param val="" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="Use the arrow keys to identify the text color. Ignore what the word says.&amp;#10;&amp;#10;A reminder will be presented at the top of the screen.&amp;#10;&amp;#10;Press space to start.&amp;#10;" valType="str" updates="constant" name="text"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
        <Param val="" valType="num" updates="constant" name="wrapWidth"/>
      </TextComponent>
      <KeyboardComponent name="block_resp" plugin="None">
        <Param val="'space'" valType="list" updates="constant" name="allowedKeys"/>
        <Param val="" valType="str" updates="constant" name="correctAns"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="True" valType="bool" updates="constant" name="discard previous"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="True" valType="bool" updates="constant" name="forceEndRoutine"/>
        <Param val="block_resp" valType="code" updates="None" name="name"/>
        <Param val="press" valType="str" updates="constant" name="registerOn"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="last key" valType="str" updates="constant" name="store"/>
        <Param val="False" valType="bool" updates="constant" name="storeCorrect"/>
        <Param val="True" valType="bool" updates="constant" name="syncScreenRefresh"/>
      </KeyboardComponent>
      <TextComponent name="instrText_4" plugin="None">
        <Param val="white" valType="str" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="" valType="str" updates="constant" name="flip"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.03" valType="code" updates="constant" name="letterHeight"/>
        <Param val="instrText_4" valType="code" updates="None" name="name"/>
        <Param val="1" valType="code" updates="constant" name="opacity"/>
        <Param val="0" valType="code" updates="constant" name="ori"/>
        <Param val="(0, 0.4)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="$Instructions + '\nIdentify the COLOR of the text. Ignore what the word says'" valType="str" updates="set every repeat" name="text"/>
        <Param val="height" valType="str" updates="None" name="units"/>
        <Param val="" valType="code" updates="constant" name="wrapWidth"/>
      </TextComponent>
    </Routine>
    <Routine name="trial">
      <RoutineSettingsComponent name="trial" plugin="None">
        <Param val="none" valType="str" updates="None" name="backgroundFit"/>
        <Param val="" valType="str" updates="None" name="backgroundImg"/>
        <Param val="$[0,0,0]" valType="color" updates="None" name="color"/>
        <Param val="rgb" valType="str" updates="None" name="colorSpace"/>
        <Param val="" valType="str" updates="constant" name="desc"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="trial" valType="code" updates="None" name="name"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="constant" name="skipIf"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="False" valType="bool" updates="None" name="useWindowParams"/>
      </RoutineSettingsComponent>
      <ImageComponent name="background_3" plugin="None">
        <Param val="center" valType="str" updates="constant" name="anchor"/>
        <Param val="$[1,1,1]" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="False" valType="bool" updates="constant" name="flipHoriz"/>
        <Param val="False" valType="bool" updates="constant" name="flipVert"/>
        <Param val="background.png" valType="file" updates="constant" name="image"/>
        <Param val="linear" valType="str" updates="constant" name="interpolate"/>
        <Param val="" valType="str" updates="constant" name="mask"/>
        <Param val="background_3" valType="code" updates="None" name="name"/>
        <Param val="" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="(2, 1.5)" valType="list" updates="constant" name="size"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="128" valType="num" updates="constant" name="texture resolution"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
      </ImageComponent>
      <TextComponent name="stim" plugin="None">
        <Param val="$wordColor" valType="str" updates="set every repeat" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="" valType="str" updates="constant" name="flip"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.15" valType="code" updates="constant" name="letterHeight"/>
        <Param val="stim" valType="code" updates="None" name="name"/>
        <Param val="1" valType="code" updates="constant" name="opacity"/>
        <Param val="0" valType="code" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.5" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="$word" valType="str" updates="set every repeat" name="text"/>
        <Param val="height" valType="str" updates="None" name="units"/>
        <Param val="" valType="code" updates="constant" name="wrapWidth"/>
      </TextComponent>
      <KeyboardComponent name="resp" plugin="None">
        <Param val="'left','down','right'" valType="list" updates="constant" name="allowedKeys"/>
        <Param val="$corrAns" valType="str" updates="constant" name="correctAns"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="True" valType="bool" updates="constant" name="discard previous"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="True" valType="bool" updates="constant" name="forceEndRoutine"/>
        <Param val="resp" valType="code" updates="None" name="name"/>
        <Param val="press" valType="str" updates="constant" name="registerOn"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val=".5" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="last key" valType="str" updates="constant" name="store"/>
        <Param val="True" valType="bool" updates="constant" name="storeCorrect"/>
        <Param val="True" valType="bool" updates="constant" name="syncScreenRefresh"/>
      </KeyboardComponent>
      <TextboxComponent name="trial_counter" plugin="None">
        <Param val="center" valType="str" updates="constant" name="alignment"/>
        <Param val="center" valType="str" updates="constant" name="anchor"/>
        <Param val="True" valType="bool" updates="constant" name="autoLog"/>
        <Param val="False" valType="bool" updates="constant" name="bold"/>
        <Param val="black" valType="color" updates="constant" name="borderColor"/>
        <Param val="2" valType="num" updates="constant" name="borderWidth"/>
        <Param val="black" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="False" valType="bool" updates="constant" name="editable"/>
        <Param val="white" valType="color" updates="constant" name="fillColor"/>
        <Param val="False" valType="bool" updates="constant" name="flipHoriz"/>
        <Param val="False" valType="bool" updates="constant" name="flipVert"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="False" valType="bool" updates="constant" name="italic"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.05" valType="num" updates="constant" name="letterHeight"/>
        <Param val="1.0" valType="num" updates="constant" name="lineSpacing"/>
        <Param val="trial_counter" valType="code" updates="None" name="name"/>
        <Param val="0.8" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="visible" valType="str" updates="constant" name="overflow"/>
        <Param val="0" valType="num" updates="constant" name="padding"/>
        <Param val="Type here..." valType="str" updates="constant" name="placeholder"/>
        <Param val="(0, -.4)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="(0.5, 0.1)" valType="list" updates="constant" name="size"/>
        <Param val="" valType="list" updates="constant" name="speechPoint"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="$'Block ' + str(blocks.thisN+1) + '/' + str(blocks.nTotal)+'.   Trial ' + str(trials.thisN+1) +'/' +str(trials.nTotal)" valType="str" updates="set every repeat" name="text"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
      </TextboxComponent>
      <TextComponent name="instrText_2" plugin="None">
        <Param val="white" valType="str" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="" valType="str" updates="constant" name="flip"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.03" valType="code" updates="constant" name="letterHeight"/>
        <Param val="instrText_2" valType="code" updates="None" name="name"/>
        <Param val="1" valType="code" updates="constant" name="opacity"/>
        <Param val="0" valType="code" updates="constant" name="ori"/>
        <Param val="(0, 0.4)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="$Instructions + '\nIdentify the COLOR of the text. Ignore what the word says'" valType="str" updates="set every repeat" name="text"/>
        <Param val="height" valType="str" updates="None" name="units"/>
        <Param val="" valType="code" updates="constant" name="wrapWidth"/>
      </TextComponent>
    </Routine>
    <Routine name="Thanks">
      <RoutineSettingsComponent name="Thanks" plugin="None">
        <Param val="none" valType="str" updates="None" name="backgroundFit"/>
        <Param val="" valType="str" updates="None" name="backgroundImg"/>
        <Param val="$[0,0,0]" valType="color" updates="None" name="color"/>
        <Param val="rgb" valType="str" updates="None" name="colorSpace"/>
        <Param val="" valType="str" updates="constant" name="desc"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="Thanks" valType="code" updates="None" name="name"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="constant" name="skipIf"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="False" valType="bool" updates="None" name="useWindowParams"/>
      </RoutineSettingsComponent>
      <ImageComponent name="background_5" plugin="None">
        <Param val="center" valType="str" updates="constant" name="anchor"/>
        <Param val="$[1,1,1]" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="False" valType="bool" updates="constant" name="flipHoriz"/>
        <Param val="False" valType="bool" updates="constant" name="flipVert"/>
        <Param val="background.png" valType="file" updates="constant" name="image"/>
        <Param val="linear" valType="str" updates="constant" name="interpolate"/>
        <Param val="" valType="str" updates="constant" name="mask"/>
        <Param val="background_5" valType="code" updates="None" name="name"/>
        <Param val="" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="(2, 1.5)" valType="list" updates="constant" name="size"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="128" valType="num" updates="constant" name="texture resolution"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
      </ImageComponent>
      <CodeComponent name="calc_acc" plugin="None">
        <Param val="" valType="extendedCode" updates="constant" name="Before Experiment"/>
        <Param val="" valType="extendedCode" updates="constant" name="Before JS Experiment"/>
        <Param val="" valType="extendedCode" updates="constant" name="Begin Experiment"/>
        <Param val="" valType="extendedCode" updates="constant" name="Begin JS Experiment"/>
        <Param val="english_accuracy_percent = Number.parseInt(((util.sum(english_accuracy) / english_accuracy.length) * 100));&amp;#10;maori_accuracy_percent = Number.parseInt(((util.sum(maori_accuracy) / maori_accuracy.length) * 100));&amp;#10;psychoJS.experiment.addData(&quot;english_accuracy_percent&quot;, english_accuracy_percent);&amp;#10;psychoJS.experiment.addData(&quot;maori_accuracy_percent &quot;, maori_accuracy_percent);&amp;#10;endfb = ((((&quot;You scored &quot; + english_accuracy_percent.toString()) + &quot;% correct for english words\nYou scored &quot;) + maori_accuracy_percent.toString()) + &quot;% correct for maori words&quot;);&amp;#10;" valType="extendedCode" updates="constant" name="Begin JS Routine"/>
        <Param val="# calculate percent correct per condition&amp;#10;english_accuracy_percent = int((sum(english_accuracy)/len(english_accuracy))*100)&amp;#10;maori_accuracy_percent = int((sum(maori_accuracy)/len(maori_accuracy))*100)&amp;#10;&amp;#10;#store averages to data file&amp;#10;thisExp.addData('english_accuracy_percent', english_accuracy_percent)&amp;#10;thisExp.addData('maori_accuracy_percent ', maori_accuracy_percent )&amp;#10;&amp;#10;# set up the feedback text&amp;#10;endfb = 'You scored ' + str(english_accuracy_percent) + '% correct for english words\nYou scored ' + str(maori_accuracy_percent) + '% correct for maori words'" valType="extendedCode" updates="constant" name="Begin Routine"/>
        <Param val="Auto-&gt;JS" valType="str" updates="None" name="Code Type"/>
        <Param val="" valType="extendedCode" updates="constant" name="Each Frame"/>
        <Param val="" valType="extendedCode" updates="constant" name="Each JS Frame"/>
        <Param val="" valType="extendedCode" updates="constant" name="End Experiment"/>
        <Param val="" valType="extendedCode" updates="constant" name="End JS Experiment"/>
        <Param val="" valType="extendedCode" updates="constant" name="End JS Routine"/>
        <Param val="" valType="extendedCode" updates="constant" name="End Routine"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="calc_acc" valType="code" updates="None" name="name"/>
      </CodeComponent>
      <TextComponent name="thanksText" plugin="None">
        <Param val="white" valType="str" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="" valType="str" updates="constant" name="flip"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.05" valType="code" updates="constant" name="letterHeight"/>
        <Param val="thanksText" valType="code" updates="None" name="name"/>
        <Param val="1" valType="code" updates="constant" name="opacity"/>
        <Param val="0" valType="code" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="$endfb" valType="str" updates="set every repeat" name="text"/>
        <Param val="height" valType="str" updates="None" name="units"/>
        <Param val="" valType="code" updates="constant" name="wrapWidth"/>
      </TextComponent>
    </Routine>
    <Routine name="feedback">
      <RoutineSettingsComponent name="feedback" plugin="None">
        <Param val="none" valType="str" updates="None" name="backgroundFit"/>
        <Param val="" valType="str" updates="None" name="backgroundImg"/>
        <Param val="$[0,0,0]" valType="color" updates="None" name="color"/>
        <Param val="rgb" valType="str" updates="None" name="colorSpace"/>
        <Param val="" valType="str" updates="constant" name="desc"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="feedback" valType="code" updates="None" name="name"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="constant" name="skipIf"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="" valType="code" updates="constant" name="stopVal"/>
        <Param val="False" valType="bool" updates="None" name="useWindowParams"/>
      </RoutineSettingsComponent>
      <ImageComponent name="background_4" plugin="None">
        <Param val="center" valType="str" updates="constant" name="anchor"/>
        <Param val="$[1,1,1]" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="False" valType="bool" updates="constant" name="flipHoriz"/>
        <Param val="False" valType="bool" updates="constant" name="flipVert"/>
        <Param val="background.png" valType="file" updates="constant" name="image"/>
        <Param val="linear" valType="str" updates="constant" name="interpolate"/>
        <Param val="" valType="str" updates="constant" name="mask"/>
        <Param val="background_4" valType="code" updates="None" name="name"/>
        <Param val="" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="(2, 1.5)" valType="list" updates="constant" name="size"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="1" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="128" valType="num" updates="constant" name="texture resolution"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
      </ImageComponent>
      <CodeComponent name="code" plugin="None">
        <Param val="" valType="extendedCode" updates="constant" name="Before Experiment"/>
        <Param val="" valType="extendedCode" updates="constant" name="Before JS Experiment"/>
        <Param val="english_accuracy = []&amp;#10;maori_accuracy = []" valType="extendedCode" updates="constant" name="Begin Experiment"/>
        <Param val="english_accuracy = [];&amp;#10;maori_accuracy = [];&amp;#10;" valType="extendedCode" updates="constant" name="Begin JS Experiment"/>
        <Param val="if (resp.corr) {&amp;#10;    fb = &quot;Correct!&quot;;&amp;#10;    fbcol = &quot;green&quot;;&amp;#10;} else {&amp;#10;    fb = &quot;Incorrect&quot;;&amp;#10;    fbcol = &quot;red&quot;;&amp;#10;}&amp;#10;if ((language === &quot;english&quot;)) {&amp;#10;    english_accuracy.push(resp.corr);&amp;#10;}&amp;#10;if ((language === &quot;maori&quot;)) {&amp;#10;    maori_accuracy.push(resp.corr);&amp;#10;}&amp;#10;" valType="extendedCode" updates="constant" name="Begin JS Routine"/>
        <Param val="if resp.corr:&amp;#10;    fb = 'Correct!'&amp;#10;    fbcol = 'green'&amp;#10;else:&amp;#10;    fb = 'Incorrect'&amp;#10;    fbcol = 'red'&amp;#10;&amp;#10;# track accuracy for each condition by adding a 1 or 0 to a list&amp;#10;if language == 'english':&amp;#10;    english_accuracy.append(resp.corr)&amp;#10;if language == 'maori':&amp;#10;    maori_accuracy.append(resp.corr)&amp;#10;" valType="extendedCode" updates="constant" name="Begin Routine"/>
        <Param val="Auto-&gt;JS" valType="str" updates="None" name="Code Type"/>
        <Param val="" valType="extendedCode" updates="constant" name="Each Frame"/>
        <Param val="" valType="extendedCode" updates="constant" name="Each JS Frame"/>
        <Param val="" valType="extendedCode" updates="constant" name="End Experiment"/>
        <Param val="" valType="extendedCode" updates="constant" name="End JS Experiment"/>
        <Param val="" valType="extendedCode" updates="constant" name="End JS Routine"/>
        <Param val="" valType="extendedCode" updates="constant" name="End Routine"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="code" valType="code" updates="None" name="name"/>
      </CodeComponent>
      <TextboxComponent name="fbtxt" plugin="None">
        <Param val="center" valType="str" updates="constant" name="alignment"/>
        <Param val="center" valType="str" updates="constant" name="anchor"/>
        <Param val="True" valType="bool" updates="constant" name="autoLog"/>
        <Param val="False" valType="bool" updates="constant" name="bold"/>
        <Param val="None" valType="color" updates="constant" name="borderColor"/>
        <Param val="2" valType="num" updates="constant" name="borderWidth"/>
        <Param val="$fbcol" valType="color" updates="set every repeat" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="False" valType="bool" updates="constant" name="editable"/>
        <Param val="None" valType="color" updates="constant" name="fillColor"/>
        <Param val="False" valType="bool" updates="constant" name="flipHoriz"/>
        <Param val="False" valType="bool" updates="constant" name="flipVert"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="False" valType="bool" updates="constant" name="italic"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.05" valType="num" updates="constant" name="letterHeight"/>
        <Param val="1.0" valType="num" updates="constant" name="lineSpacing"/>
        <Param val="fbtxt" valType="code" updates="None" name="name"/>
        <Param val="" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="visible" valType="str" updates="constant" name="overflow"/>
        <Param val="0" valType="num" updates="constant" name="padding"/>
        <Param val="Type here..." valType="str" updates="constant" name="placeholder"/>
        <Param val="(0, 0)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="(0.5, 0.5)" valType="list" updates="constant" name="size"/>
        <Param val="" valType="list" updates="constant" name="speechPoint"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="1.0" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="$fb" valType="str" updates="set every repeat" name="text"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
      </TextboxComponent>
      <TextboxComponent name="trial_counter_2" plugin="None">
        <Param val="center" valType="str" updates="constant" name="alignment"/>
        <Param val="center" valType="str" updates="constant" name="anchor"/>
        <Param val="True" valType="bool" updates="constant" name="autoLog"/>
        <Param val="False" valType="bool" updates="constant" name="bold"/>
        <Param val="black" valType="color" updates="constant" name="borderColor"/>
        <Param val="2" valType="num" updates="constant" name="borderWidth"/>
        <Param val="black" valType="color" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="False" valType="bool" updates="constant" name="editable"/>
        <Param val="white" valType="color" updates="constant" name="fillColor"/>
        <Param val="False" valType="bool" updates="constant" name="flipHoriz"/>
        <Param val="False" valType="bool" updates="constant" name="flipVert"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="False" valType="bool" updates="constant" name="italic"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.05" valType="num" updates="constant" name="letterHeight"/>
        <Param val="1.0" valType="num" updates="constant" name="lineSpacing"/>
        <Param val="trial_counter_2" valType="code" updates="None" name="name"/>
        <Param val="0.8" valType="num" updates="constant" name="opacity"/>
        <Param val="0" valType="num" updates="constant" name="ori"/>
        <Param val="visible" valType="str" updates="constant" name="overflow"/>
        <Param val="0" valType="num" updates="constant" name="padding"/>
        <Param val="Type here..." valType="str" updates="constant" name="placeholder"/>
        <Param val="(0, -.4)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="(0.5, 0.1)" valType="list" updates="constant" name="size"/>
        <Param val="" valType="list" updates="constant" name="speechPoint"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="1" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="$'Block ' + str(blocks.thisN+1) + '/' + str(blocks.nTotal)+'.   Trial ' + str(trials.thisN+1) +'/' +str(trials.nTotal)" valType="str" updates="set every repeat" name="text"/>
        <Param val="from exp settings" valType="str" updates="None" name="units"/>
      </TextboxComponent>
      <TextComponent name="instrText_3" plugin="None">
        <Param val="white" valType="str" updates="constant" name="color"/>
        <Param val="rgb" valType="str" updates="constant" name="colorSpace"/>
        <Param val="1" valType="num" updates="constant" name="contrast"/>
        <Param val="False" valType="bool" updates="None" name="disabled"/>
        <Param val="" valType="code" updates="None" name="durationEstim"/>
        <Param val="" valType="str" updates="constant" name="flip"/>
        <Param val="Arial" valType="str" updates="constant" name="font"/>
        <Param val="LTR" valType="str" updates="None" name="languageStyle"/>
        <Param val="0.03" valType="code" updates="constant" name="letterHeight"/>
        <Param val="instrText_3" valType="code" updates="None" name="name"/>
        <Param val="1" valType="code" updates="constant" name="opacity"/>
        <Param val="0" valType="code" updates="constant" name="ori"/>
        <Param val="(0, 0.4)" valType="list" updates="constant" name="pos"/>
        <Param val="True" valType="bool" updates="None" name="saveStartStop"/>
        <Param val="" valType="code" updates="None" name="startEstim"/>
        <Param val="time (s)" valType="str" updates="None" name="startType"/>
        <Param val="0.0" valType="code" updates="None" name="startVal"/>
        <Param val="duration (s)" valType="str" updates="None" name="stopType"/>
        <Param val="1" valType="code" updates="constant" name="stopVal"/>
        <Param val="True" valType="bool" updates="None" name="syncScreenRefresh"/>
        <Param val="$Instructions + '\nIdentify the COLOR of the text. Ignore what the word says'" valType="str" updates="set every repeat" name="text"/>
        <Param val="height" valType="str" updates="None" name="units"/>
        <Param val="" valType="code" updates="constant" name="wrapWidth"/>
      </TextComponent>
    </Routine>
  </Routines>
  <Flow>
    <Routine name="intro"/>
    <LoopInitiator loopType="TrialHandler" name="blocks">
      <Param name="Selected rows" updates="None" val="" valType="str"/>
      <Param name="conditions" updates="None" val="None" valType="str"/>
      <Param name="conditionsFile" updates="None" val="$'group' + expInfo['group'] + '.xlsx'" valType="file"/>
      <Param name="endPoints" updates="None" val="[0, 1]" valType="num"/>
      <Param name="isTrials" updates="None" val="True" valType="bool"/>
      <Param name="loopType" updates="None" val="sequential" valType="str"/>
      <Param name="nReps" updates="None" val="1" valType="num"/>
      <Param name="name" updates="None" val="blocks" valType="code"/>
      <Param name="random seed" updates="None" val="" valType="code"/>
    </LoopInitiator>
    <Routine name="instructions"/>
    <LoopInitiator loopType="TrialHandler" name="trials">
      <Param name="Selected rows" updates="None" val="" valType="str"/>
      <Param name="conditions" updates="None" val="None" valType="str"/>
      <Param name="conditionsFile" updates="None" val="$language + '.xlsx'" valType="file"/>
      <Param name="endPoints" updates="None" val="[0, 1]" valType="num"/>
      <Param name="isTrials" updates="None" val="True" valType="bool"/>
      <Param name="loopType" updates="None" val="random" valType="str"/>
      <Param name="nReps" updates="None" val="1" valType="num"/>
      <Param name="name" updates="None" val="trials" valType="code"/>
      <Param name="random seed" updates="None" val="" valType="code"/>
    </LoopInitiator>
    <Routine name="trial"/>
    <Routine name="feedback"/>
    <LoopTerminator name="trials"/>
    <LoopTerminator name="blocks"/>
    <Routine name="Thanks"/>
  </Flow>
</PsychoPy2experiment>
'''

In [8]:
messages = [{"role": "user", "content": f"{system_prompt}\n Input: {query}"}]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

In [11]:
outputs = model.generate(inputs, max_new_tokens=200)
tokenizer.batch_decode(outputs)[0]

error detectedA: torch.Size([16600, 3072]), B: torch.Size([9216, 3072]), C: (16600, 9216); (lda, ldb, ldc): (c_int(16600), c_int(9216), c_int(16600)); (m, n, k): (c_int(16600), c_int(9216), c_int(3072))


Exception: cublasLt ran into an error!